# Armani SageMaker instance 

In [2]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
# from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Matplotlib is building the font cache; this may take a moment.


ImportError: cannot import name 'csv_serializer' from 'sagemaker.predictor' (/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/predictor.py)

In [ ]:
bucket_name = 'cestino-s3'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

The part until now is equal to the [tutorial](https://aws.amazon.com/it/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/).

In [ ]:
try:
  urllib.request.urlretrieve("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani.csv')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./products.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Now that we have loaded into model_data the CSV...

In [ ]:
print(model_data) # ...and after a bit of debugging...
print(type(model_data)) # ...and compensation of the absurd leak of documentation...

...we start splitting data into the data for training (in this case 70% of the total) an data for testing (the rest).

In [ ]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

We should first get what [random_state](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) is.
Well, it's just a randomizer I guess.

In [1]:
pd.concat(
    [
        train_data['y_yes'],
        train_data.drop(
            ['y_no', 'y_yes'],
            axis=1
        )
    ], axis=1
).to_csv(
    'train.csv',
    index=False,
    header=False
)
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(
        prefix, 'train/train.csv'
    )
).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)

NameError: name 'pd' is not defined